# 11.7 달 표면 착륙선의 제어 -구현-


리스트11.8 필요한 라이브러리 설치

In [ ]:
!apt-get update
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines==2.5.1 box2d box2d-kengz pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!pip install tensorflow==1.14

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [84.8 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.laun

리스트11.9 라이브러리 사용

In [ ]:
import os
import io
import glob
import base64

import gym
from gym.wrappers import Monitor

import numpy as np

from stable_baselines.deepq.policies import MlpPolicy  # 2층의 신경망
from stable_baselines.common.vec_env import DummyVecEnv  # 벡터화 환경
from stable_baselines import DQN
from stable_baselines.common.vec_env import VecVideoRecorder

from IPython import display as ipythondisplay
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

리스트11.10 달 표면 착륙선의 환경을 설정

In [ ]:
def env_func():  # 환경을 만드는 함수
    return gym.make("LunarLander-v2")

env_vec = DummyVecEnv([env_func])  # 환경의 벡터화

AttributeError: ignored

리스트11.11 모델을 평가하기 위한 함수의 준비

In [ ]:
def evaluate(env, model, n_step=10000, n_ave=100):

  epi_rewards = [0.0]  # 에피소드별 보상을 저장
  states = env.reset()

  for i in range(n_step):
      action, _h = model.predict(states)  # _h는 RNN에서 사용(이번은 사용하지 않는다)
      states, rewards, dones, info = env.step(action)
      
      epi_rewards[-1] += rewards[0]  # 마지막의 요소에 누적
      if dones[0]:  # 에피소드 종료 시
          states = env.reset()
          epi_rewards.append(0.0)  # 다음 에피소드의 보상

  ave_reward = round(np.average(epi_rewards[:n_ave]), 2)  # 첫 100 에피소드에서 보상의 평균을 취한다
  return (ave_reward, len(epi_rewards))

리스트11.12 동영상으로 표시하기 위한 함수의 준비

In [ ]:
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

def show_video(video_dir):
  video_list = glob.glob(video_dir+"/*.mp4")
  if len(video_list) > 0:
    mp4 = video_list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))

리스트11.13 DQN 모델을 설정하고 훈련 전에 평가한다


In [ ]:
env = VecVideoRecorder(env_vec, video_folder="videos_before_train/",  # 동영상 기록의 설정
                            record_video_trigger=lambda step: step == 0, video_length=500,
                            name_prefix="")

model = DQN(MlpPolicy, env, verbose=0)  # DQN의 설정

ave_reward, n_episode = evaluate(env, model, n_step=10000, n_ave=100)  # 모델의 평가
print("ave_reward:", ave_reward, "n_episode:", n_episode)







Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ave_reward: -148.35 n_episode: 139


리스트11.14 동작의 확인

In [ ]:
show_video("videos_before_train")

리스트11.15 모델의 훈련

In [ ]:
trained_model = DQN(MlpPolicy, env_vec, verbose=0)  # 모델의 초기화

trained_model.learn(total_timesteps=100000)  # 모델의 훈련
trained_model.save("lunar_lander_control")  # 모델의 저장

리스트11.16 훈련된 모델의 평가

In [ ]:
env = VecVideoRecorder(env_vec, video_folder="videos_after_train/",  # 동영상 기록의 설정
                            record_video_trigger=lambda step: step == 0, video_length=500,
                            name_prefix="")

ave_reward, n_episode = evaluate(env, trained_model, n_step=10000, n_ave=100)  # 모델의 평가
print("ave_reward:", ave_reward, "n_episode:", n_episode)

Saving video to  /content/videos_after_train/-step-253-to-step-753.mp4
Saving video to  /content/videos_after_train/-step-920-to-step-1420.mp4
Saving video to  /content/videos_after_train/-step-1854-to-step-2354.mp4
Saving video to  /content/videos_after_train/-step-2477-to-step-2977.mp4
Saving video to  /content/videos_after_train/-step-3254-to-step-3754.mp4
Saving video to  /content/videos_after_train/-step-3951-to-step-4451.mp4
Saving video to  /content/videos_after_train/-step-4778-to-step-5278.mp4
Saving video to  /content/videos_after_train/-step-5778-to-step-6278.mp4
Saving video to  /content/videos_after_train/-step-6624-to-step-7124.mp4
Saving video to  /content/videos_after_train/-step-7624-to-step-8124.mp4
Saving video to  /content/videos_after_train/-step-8314-to-step-8814.mp4
Saving video to  /content/videos_after_train/-step-8925-to-step-9425.mp4
ave_reward: 159.91 n_episode: 15


리스트11.17 동작의 확인

In [ ]:
show_video("videos_after_train/")